# Introduction to Pandas

## 1. What is pandas?
## Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language.
## 2. Why is pandas so popular?
## Pandas is the *de facto* standard in data analysis and data manipulation in Python. In the backend it stores data as NumPy arrays, which in turns gives C-like performance whilst mantaining code simplicity.
## 3. Any drawback to be aware off?
## Pandas is very memory consuming, if a .csv file is X MB, you should expect pandas to take around 5-10 X memory in your RAM. This becomes a problem if you want to load several GB worth of data. Hence, scalability is not ideal and other technologies should be considered to ingest GB or TB sized data streams


# Pandas Series

## Pandas series are the simplest structure available in Pandas. They, describe a dataset labelled by a index and a single column. For instance, this could be a time-series of a stock price.

In [1]:
import pandas as pd
import numpy as np
# We create a date range
dates = pd.date_range('20200101', periods=200)
# And some random values for a stock
stock_value=100+np.random.normal(0,1,200)

df = pd.Series(stock_value, index=dates, name='stock_price')

print(df)

2020-01-01    101.588572
2020-01-02    101.561367
2020-01-03    100.171172
2020-01-04     98.651032
2020-01-05     97.318439
                 ...    
2020-07-14     99.717189
2020-07-15     99.950478
2020-07-16     99.864886
2020-07-17    100.852199
2020-07-18    101.443511
Freq: D, Name: stock_price, Length: 200, dtype: float64


###  Pandas series offer a number of functionalities, like plotting the data in a very straightforward manner (for more info visit https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.html)

In [ ]:
df.plot(title="Stock value",grid=True,legend=True)

### Another useful feature is to be able to display summary statistics of our data set

In [ ]:
df.describe()

### We can also display first/last n rows using ```head(n)``` or ```tail(n)```

In [ ]:
df.head(5)

In [ ]:
df.tail(2)

# Pandas DataFrames
### Before advancing further, let's introduce the concept of a dataframe. A Pandas DataFrame is a collection of Series labelled by the same index, e.g. it could be a collection of Series of Stock prices labelled by their timestamp. Let's have a look at some examples looking at the```yahoo_fin``` library


In [ ]:
import yahoo_fin.stock_info as si

AAPL = si.get_data("aapl") # gets Apple's data

In [ ]:
type(AAPL) # Check data typr

In [ ]:
AAPL

### As mentioned, before ```yahoo_fin``` provides us with a dataframe directly with a number of columns, that can then be accessed by typing ```dataframe.columns```, and likewise for the index, which canbe accessed using  ```dataframe.index```

In [ ]:
AAPL.columns

In [ ]:
AAPL.index


# Storing and loading dataframes
### Pandas offers a number of format compatibility like ```dataframe.to_csv``` .csv,  ```dataframe.to_pickle``` .pickle etc. 

In [ ]:
AAPL.to_csv('AAPL.csv',index=True)
pd.read_csv('AAPL.csv',index_col=0)

In [ ]:
AAPL.to_pickle('AAPL.pickle')
pd.read_pickle('AAPL.pickle')

# Slicing DataFrames

### Perhaps, not all the information in a dataframe is relevant to us and often case we want to either remove part of it or transform it. Let us start by selecting a single column from the dataframe above. There are a numbers of ways to do this operation:

In [ ]:
# Let's assume we are interested in adjclose column in AAPL dataframe
AAPL_adjclose=AAPL.adjclose # we can type explicitly the name of the column
print(type(AAPL_adjclose))
AAPL_adjclose

In [ ]:
# Equivalently we can use 
AAPL_adjclose=AAPL['adjclose'] # we can type explicitly the name of the column
print(type(AAPL_adjclose))
AAPL_adjclose

### ***Remark***:  Slicing a dataframe into a single column returns  a Series!

# Creating new columns

In [ ]:
AAPL['log_stock_close_adj']=np.log(AAPL.adjclose)

In [ ]:
AAPL.head()

# Some useful functions
### Pandas provides mean, std,... and a number of functions that can be applied to both axis

In [ ]:
AAPL.mean(axis=0)

In [ ]:
AAPL.std(axis=0)

# Rolling windows

### In time-series it is useful to apply a function on observation windows of size $n$. We can do so using ```dataframe.rolling(n)```

In [ ]:
# Let#s compute the 10 day moving average
AAPL.adjclose.rolling(10).mean()

# Diff and Shift
### ```dataframe.diff(n)``` computes the difference with respect to the tow n postitions below. Shift ``dataframe.shift(n)``` displaces the dataframe $n$ rows down. Using this functions together we can easily compute returns

In [ ]:
i=1
AAPL['daily_return'] = (AAPL.adjclose.diff(i) - AAPL.adjclose.diff(i - 1)) / AAPL.adjclose.shift(i)

In [ ]:
AAPL.head()

# Slicing using  ```loc ```
### Another approach to slice a dataframe is to use ```dataframe.loc(index,columns)```. This allows to retrieve one or more columns. Let's look at some examples

In [ ]:
#We try to obtain adjclose column as before
AAPL_adjclose=AAPL.loc[:,'adjclose']
print(type(AAPL_adjclose))
AAPL_adjclose

In [ ]:
#Let's slice high and low
AAPL_high_low=AAPL.loc[:,['low','high']]
print(type(AAPL_high_low))
AAPL_high_low

### ***Remark***: When multiple columns are sliced, the returned object is again a pandas dataframe

### It is also posible to slice index and columns at the same time. Let's assume we are only interested in 2020 high and low data. And let's use  ```plot() ``` and  ```describe() ``` in the outcome 

In [ ]:
import datetime
#I strongly recommend to have a look at datetime module as you will be often dealing with this when using time-series data 
datetime.datetime(2020, 5, 17)

In [ ]:
import datetime
# We select just 2019 dates
dates_2019=AAPL.index[(AAPL.index>=datetime.datetime(2019, 1, 1)) & (AAPL.index<datetime.datetime(2020, 1, 1))]
#We slice the dataframe
AAPL_high_low=AAPL.loc[dates_2019,['low','high']]
print(type(AAPL_high_low))
#Plot the data
AAPL_high_low.plot(grid=True,title='AAPL 2020 daily high and low')

In [ ]:
print(AAPL_high_low.describe())

# Slicing using  ```iloc```
### in order to use loc one needs to specify the index and column precisely. Using .iloc we can treat the Dataframe as if it was a numpy array and slice using integers as usual

In [ ]:
AAPL.iloc[-100:,[0,2,3]]# gives last 100 rows for columns # 0 2 3

### ***Remark***: There is also the option to cast a dataframe or series into a numpy array using  ```.values ```. Note that this operation will get rid of the index column. 

In [ ]:
AAPL.values

In [ ]:
AAPL.low.values

# ```at``` and ```iat```: retrieving single values
### When retrieving a single value from a dataframe it is much more efficient to use ```at```/```iat``` instead of ```loc```/```iloc```

In [ ]:
%timeit AAPL.iloc[0,0]
%timeit AAPL.iat[0,0]

# Boolean slicing

### We can also slice a dataframe whenever a boolean condition is satisfied

In [ ]:
# Filter data by volume
AAPL.loc[AAPL.volume<10000000,]

# Carefull with copies!!!

In [ ]:
df2=AAPL
print(AAPL.iloc[0,0])
df2.iloc[0,0]=0
print(AAPL.iloc[0,0])

# Iterating over Dataframes
### Sometime our analysis requires to go row by row in a dataframe to perform a numerical operation. To do this we can use ```iterrows``` or ```itertuples``` . 

In [ ]:
import time
high_low_mid=np.zeros(len(AAPL))
index=0
start_time=time.time()
for row in AAPL.iterrows():
    high_low_mid[index]=0.5*(row[1].high+row[1].low)
    index+=1
print("iterrows took", time.time()-start_time,"seconds")

In [ ]:
print(high_low_mid)

In [ ]:
high_low_mid=np.zeros(len(AAPL))
index=0
start_time=time.time()
for row in AAPL.itertuples():    
    high_low_mid[index]=0.5*(row.high+row.low)
    index+=1
print("itertuples took", time.time()-start_time,"seconds")

In [ ]:
print(high_low_mid)

### If going row by row is necessary (as we will see in a minute, vectorisation is always preferred), most of the time `itertuples` is much more efficient 

# Vectorisation in DataFrame Columns and slices
### As mentioned in the beginning of the session, internally Pandas stores the data as numpy arrays. Hence, we can make use of vectorisation to speed up computations

In [ ]:
%timeit high_low_mid=0.5*(AAPL.high+AAPL.low)

In [ ]:
high_low_mid=0.5*(AAPL.high+AAPL.low)
high_low_mid.values

In [ ]:
#We can apply the same pinciple if we want a slice corresponding to 2019 data

dates_2019=AAPL.index[(AAPL.index>=datetime.datetime(2019, 1, 1)) & (AAPL.index<datetime.datetime(2020, 1, 1))]

%timeit high_low_mid_2019=0.5*(AAPL.high[dates_2019]+AAPL.low[dates_2019])# gives some performance improvement

%timeit high_low_mid_2019=0.5*(AAPL.loc[dates_2019,'high']+AAPL.loc[dates_2019,'low'])



# Complex functions and  ```apply```
### Using ```apply``` we can vectorise any user-defined function that supports vectorisation and apply it indexwise (axis=1) or columnwise (axis=0). One can further optimize the method by setting ```raw=False``` which will make assume objects to be numpy arrays internally

In [ ]:
AAPL=AAPL.loc[:,['open','high','low','close']]

In [ ]:
# Columnwuse
def my_func(x):
    # Function will be applied columnwise x represents the entire column
    return x.max() -x.min()

%timeit AAPL.apply(my_func,axis=0,raw=False)
%timeit AAPL.apply(my_func,axis=0,raw=True)

AAPL.apply(my_func,axis=0,raw=False)
    

In [ ]:
def my_func_high_low_mid(x):
    
     # Function will be applied rowise, x the entire 
    return 0.5*(x.high+x.low) #here we assume x mantains the column structure

def my_func_high_low_mid2(x):
    # Function will be applied rowise, x the entire row casted to numpy array
    return 0.5*(x[1]+x[2]) # We need to use integer slicing

%timeit AAPL.apply(my_func_high_low_mid,axis=1,raw=False)
%timeit AAPL.apply(my_func_high_low_mid2,axis=1,raw=True)

### As you can see ```raw=False``` can give dramatic performance improvements as data will be treated as a numpy array

# Join/Merge/Concatenate/Append dataframes and series

## 1 Join and Merge

### Now that we have a clear view on basic data manipulation, we can ask ourselves how can we merge data from different Stocks?

In [ ]:
list_of_symbols=['aapl','amzn','dis','msft','spy']

dict_of_df={}
for symbol in list_of_symbols:
    dict_of_df[symbol]=si.get_data(symbol) 



In [ ]:
dict_of_df['amzn']

In [ ]:
dict_of_df['aapl']

### We can merge two dataframes by index using ```merge```. By setting ```how='inner'``` we make sure that only intersecting indices will be selected and likewise setting ```left_index=True```, ```right_index=True``` we make sure that both indices are being considered 

In [ ]:
%timeit merged_df1=dict_of_df['aapl'].merge(dict_of_df['amzn'],how='inner',left_index=True, right_index=True,suffixes=('_aapl','_amzn'))

In [ ]:
merged_df1=dict_of_df['aapl'].merge(dict_of_df['amzn'],how='inner',left_index=True, right_index=True,suffixes=('_aapl','_amzn'))
merged_df1.head()

### Likewise we can perform the same operation using  ```join```

In [ ]:
%timeit merged_df2=dict_of_df['aapl'].join(dict_of_df['amzn'],how='inner',lsuffix='_aapl',rsuffix='_amzn')

In [ ]:
merged_df2=dict_of_df['aapl'].join(dict_of_df['amzn'],how='inner',lsuffix='_aapl',rsuffix='_amz')
merged_df2.head()

### ***Remark***: the difference between ```merge``` and ```join``` is that ```merge``` allows for more flexibility allowing to merge by column values as well. In general, merge performance tends to be better at the cost of providing more arguments

## 2. Append
### ```append``` allows to add additional rows to an existing dataframe

In [ ]:
AAPL.append({"open": 100,"high": 100,"low": 100,"close": 100,"adjclose": 100 ,"volume": 100, "ticker": 'AAPL'  },ignore_index=True)

In [ ]:
dict_of_df['aapl'].append(dict_of_df['amzn'])

## 3. Concat and Multi_indexing
### Concatenate allows to merge multiple df at once, but will create a multi-index/multi-column data frame

In [ ]:
df1=pd.concat(dict_of_df,axis=1)

In [ ]:
df2=pd.concat(dict_of_df,axis=0)

### This introduces multi_indexing

In [ ]:
df1.columns

In [ ]:
df1.loc[:,('msft','close')]

In [ ]:
df2.index

In [ ]:
df2.loc[('aapl'),:]

# Groupby


### Groupy allows to index data using different columns or compute summary statistics for groups wihin the data. Let's look at some option data


In [ ]:
import yfinance as yf
import numpy as np
import pandas
aapl = yf.Ticker("AMZN")

In [ ]:
pd.read_csv('AMZN_options.csv')

In [ ]:
option_data=option_data.dropna()

In [ ]:
option_data.head()

### the syntax is ```dataframe.groupby([columns])```

In [ ]:
option_data.groupby(["expiration_date","option_type"]).count()

In [ ]:
grouped_option_chain=option_data.groupby(["expiration_date","option_type","strike"]).mean()

In [ ]:
grouped_option_chain

In [ ]:
grouped_option_chain.loc[('2020-11-06','C'),:]

# Remove missing data ```dropna``` and ```interpolate```


In [ ]:
### Let's introduce some missing values and see how we can remove or interpolate values

In [ ]:
AAPL_copy=AAPL.iloc[:,:-1]
AAPL_copy.iloc[2,:]=np.nan
AAPL_copy.head()

In [ ]:
# Dropna just removes the rows that contain a NaN
AAPL_copy.dropna(axis=0)
# We can also use axis=1 to drop columns

In [ ]:
AAPL_copy.interpolate(method='linear', axis=0)

# Further reading for big data

### If you are dealing with big data is likely that pandas will consume all your memory, so other tools are preferred which use *lazy evaluation* like VAEX https://pypi.org/project/vaex/ or PySpark https://spark.apache.org/docs/latest/api/python/index.html 

### Lazy evaluation essentially means that a plan will be set to execute your operation, but it won't be actually executed until you need to retrieve a value. For instance creating a new column can be a lazy operation until we need to retrieve some value in that column.

### This approach allows to use multiprocessing to speed up calculations